# NLTK application

**1. Imports and databse connection**

In [1]:
import nltk
import sqlite3
import pandas as pd
import string
import regex as re 

import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import TweetTokenizer
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords

conn = sqlite3.connect('/Users/ograndberry/Desktop/berlin_startup_jobs_analysis/bsj_db.db')
c = conn.cursor()

**2. Get the raw data**

In [2]:
raw_data = pd.read_csv('Machine-Learning-clustering-skills/merged_jobs.csv', index_col=0)
raw_data.head()

,ID,company_name,date,description,jobtitle,source
0,1,Fatmap,2017-10-02,Role & Responsibility: \n\nYou’ll be building ...,Mobile Engineer for Outdoor Adventure Sports,Berlin Startup Jobs
1,2,AI Engine,2017-10-02,AI Engine is developing innovative machine lea...,Machine Learning,Berlin Startup Jobs
2,3,November,2017-10-02,Your mission:\n\nDevelopment of a scalable sof...,Senior Full Stack PHP Developer (f/m),Berlin Startup Jobs
3,4,Wayfair,2017-10-02,Your tasks:\n\nKeyword research and competitor...,Jr. Manager SEO UK (Onpage) (m/f),Berlin Startup Jobs
4,5,CrossEngage,2017-10-02,About CrossEngage\nCrossEngage is a cloud-base...,Data Scientist,Berlin Startup Jobs


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489 entries, 0 to 488
Data columns (total 4 columns):
id              489 non-null int64
company_name    489 non-null object
jobtitle        489 non-null object
description     489 non-null object
dtypes: int64(1), object(3)
memory usage: 15.4+ KB


**3. Import the perks, skills, and industry csv files**

In [3]:
perks_rawdata= pd.read_csv('Perks.csv')
industry_rawdata = pd.read_csv('Industry.csv')

In [4]:
perks_rawdata.head()

,group,item1,item2,item3,item4,item5,item6,item7,item8,item9,...,item13,item14,item15,item16,item17,item18,item19,item20,item21,item22
0,Free Time,Free Time,Parental Leave,Unlimited Vacation,paid time off,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Personal development,Education,Great mentors,career development,potential to develop,career development,personal learning,trainings,workshops,conferences,...,coaching,MENTORING,meetups,learn@work,external training budget,offered trainings,linkedin learning account,book allowance,shared kindles,Personal development
2,Sport,gym,Yoga,Pilates,massage,gym membership,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Flexible Schedules,Working for home,flexible work time,Flexible work hours,Flexible working hours,Flexible Schedules,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Catered Meals,Free meal,Free meals,Team lunch,team breakfast,birthday cake,Cake On Your Birthday,Learning lunch,lunches,Coffee,...,snacks,drinks,subsidized lunch,healthy lunches,bbqs,cereals,beer,brunches,cocktail nights,cocktail


**Reshape the tables**

In [5]:
#reshape to 2 to rows 
perks_rawdata = pd.melt(perks_rawdata, id_vars=['group'])
#drop null values
perks_rawdata.dropna(inplace=True)
#drop unecessary columns
perks_rawdata.drop('variable',1, inplace=True)
#make the text lower
perks_rawdata['group'] = perks_rawdata['group'].str.lower()
perks_rawdata['value'] = perks_rawdata['value'].str.lower()

In [6]:
industry_rawdata = pd.melt(industry_rawdata, id_vars=['group'])
industry_rawdata.dropna(inplace=True)
industry_rawdata.drop('variable',1, inplace=True)
industry_rawdata['group'] = industry_rawdata['group'].str.lower()
industry_rawdata['value'] = industry_rawdata['value'].str.lower()

In [7]:
#check an example outcome
perks_rawdata.head()

,group,value
0,free time,free time
1,personal development,education
2,sport,gym
3,flexible schedules,working for home
4,catered meals,free meal


**4. Create list of keywords for each category**

In [8]:
#empty list to save the 3 following lists in it
all_list = []
perks_list = perks_rawdata['value'].str.lower().str.split().tolist()
industry_list = industry_rawdata['value'].str.lower().str.split().tolist()

#join all lists 
all_list.extend(perks_list)
all_list.extend(industry_list)

**5. Use the list to tockenize the job descriptions**

In [9]:
tokenizer = MWETokenizer()
tknzr = TweetTokenizer()

In [10]:
def text_process_group(mess):
    """
    1. Lower case the input
    2. Remove punctuation expect '-'
    3. Apply custom tokenizer
    4. Return column of clean text words"""
    mess.lower()
    regex = r"[^\P{P}-]+"
    new_mess= re.sub(regex, " ", mess, 0)    
    tokenizer = MWETokenizer(all_list, separator=' ')
    token = tokenizer.tokenize(new_mess.lower().split())
    sw = [x for x in token if x not in stopwords.words('english')]
    return sw

In [11]:
#apply the customized tokenizer, it takes a bit more time
raw_data['description 2'] = raw_data['description'].apply(text_process_group)

In [12]:
raw_data.head()

,ID,company_name,date,description,jobtitle,source,description 2
0,1,Fatmap,2017-10-02,Role & Responsibility: \n\nYou’ll be building ...,Mobile Engineer for Outdoor Adventure Sports,Berlin Startup Jobs,"[role, responsibility, building, fatmap, mobil..."
1,2,AI Engine,2017-10-02,AI Engine is developing innovative machine lea...,Machine Learning,Berlin Startup Jobs,"[ai, engine, developing, innovative, machine, ..."
2,3,November,2017-10-02,Your mission:\n\nDevelopment of a scalable sof...,Senior Full Stack PHP Developer (f/m),Berlin Startup Jobs,"[mission, development, scalable, software, pla..."
3,4,Wayfair,2017-10-02,Your tasks:\n\nKeyword research and competitor...,Jr. Manager SEO UK (Onpage) (m/f),Berlin Startup Jobs,"[tasks, keyword, research, competitor, analysi..."
4,5,CrossEngage,2017-10-02,About CrossEngage\nCrossEngage is a cloud-base...,Data Scientist,Berlin Startup Jobs,"[crossengage, crossengage, cloud-based, market..."


**6. Create clusters**

In [13]:
#create dictionnaries of keywords and their respective clusters
perks_model = perks_rawdata.set_index('value').to_dict()['group']
industry_model = industry_rawdata.set_index('value').to_dict()['group']

**7. Apply cluster to harmonize**

In [14]:
perks_tagger = nltk.tag.UnigramTagger(model=perks_model)
industry_tagger = nltk.tag.UnigramTagger(model=industry_model)

In [16]:
#create new tables with the id and the description 2
perks_table = raw_data[['ID','description 2']]
industry_table = raw_data[['ID','description 2']]

In [19]:
#create the tagging fuction 

def applytagperks(word):
    tag= perks_tagger.tag(word)
    return tag

def applytagindustry(word):
    tag= industry_tagger.tag(word)
    return tag

In [20]:
#apply the tagging fuction
perks_table['tags'] = perks_table['description 2'].apply(applytagperks)
industry_table['tags'] = industry_table['description 2'].apply(applytagindustry)

In [21]:
perks_table.head()

,ID,description 2,tags
0,1,"[role, responsibility, building, fatmap, mobil...","[(role, None), (responsibility, None), (buildi..."
1,2,"[ai, engine, developing, innovative, machine, ...","[(ai, None), (engine, None), (developing, None..."
2,3,"[mission, development, scalable, software, pla...","[(mission, None), (development, None), (scalab..."
3,4,"[tasks, keyword, research, competitor, analysi...","[(tasks, None), (keyword, None), (research, No..."
4,5,"[crossengage, crossengage, cloud-based, market...","[(crossengage, None), (crossengage, None), (cl..."


**8. Clean the final tables**

In [28]:
def create_tables_for_db(input_table):
    input_table = input_table.set_index(['ID'])['tags'].apply(pd.Series).stack()
    input_table = pd.DataFrame(input_table.reset_index(level=1, drop=True))
    input_table.columns = ['combinaisons']
    input_table.index.names = ['job_id']
    #split tuples into 2 columns
    input_table = input_table['combinaisons'].apply(pd.Series)
    #rename column
    input_table.columns = ['words','input_name']
    input_table.dropna(inplace = True)
    input_table.reset_index(inplace = True)
    #when a job ad muliple keywords leading to the same category we need to avoid repetition
    input_table = input_table.drop_duplicates(['job_id','input_name']).set_index('job_id')
    output = input_table[['input_name']]
    return output

In [35]:
perks = create_tables_for_db(perks_table)

In [36]:
industry = create_tables_for_db(industry_table)

In [41]:
perks.columns = ['perk_name']
industry.columns = ['industry_name']

In [42]:
perks.head()

,perk_name
job_id,
3,personal development
3,team events
3,catered meals
3,attractive compensation
4,personal development


In [43]:
industry.head()

,industry_name
job_id,
5,marketing solutions
5,fooddelivery
15,games
32,service
37,fooddelivery


**9. Send the tables to the database**

In [28]:
perks.to_sql('Perks', conn, if_exists='replace')
industry.to_sql('Industry', conn, if_exists='replace')